In [1]:
import pandas as pd
import seaborn as sns
import os,io
import time
from collections import defaultdict

import matplotlib.pyplot as plt
#sns.set(style="darkgrid")
#sns.set(style="whitegrid")
#sns.set_style("white")
#sns.set_context("notebook")
#sns.set(style="whitegrid",font_scale=1)
import matplotlib.collections as clt
import matplotlib

import tifffile
import nrrd
import numpy as np

import time


import h5py, hdf5plugin

import tqdm
import ptitprince as pt

In [2]:
font = {
        'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 7
        }

matplotlib.rc('font', **font)

In [3]:
datasets = {
    'segment':nrrd.read('../../data/CCFv3_annotation_25.nrrd')[0],
    'brainbow':tifffile.imread('../../data/nTracer_sample.tif'),
    'fly':tifffile.imread('../../data/20161215_elav_nuc5FP_2onAEL_60minHS_3L_B_20x.czi_5Chn.tif'),
    'neurites':tifffile.imread('../../data/061022_642_17us_brain_liposomes.tif'),
    'noise16':tifffile.imread('../../data/noise.tif'),
    'bead':tifffile.imread('../../data/6.1.tif'),
    'visium':tifffile.imread('../../data/V1_Breast_Cancer_Block_A_Section_1_image.tif')[5000:20000,5000:20000],
    'fMOST':h5py.File('/home/loganaw/dev/nTracer_in_Neuroglancer/pyramid_data_format/example_VDS/1_1X_VDS.h5', 'r')['data']
}
datasets['noise8'] = datasets['noise16'].astype(np.uint8)

#    ('fMOST', '../../data/182725_04301_Zprojection.tif')
for k,v in datasets.items():
    print(k, v.shape, v.dtype)

segment (528, 320, 456) uint32
brainbow (136, 4, 512, 512) uint16
fly (97, 5, 3000, 3000) uint16
neurites (1000, 2304, 2304) uint16
noise16 (100, 1000, 1000) uint16
bead (1000, 2304, 2304) uint16
visium (15000, 15000, 3) uint8
fMOST (12288, 32768, 20480) uint16
noise8 (100, 1000, 1000) uint8


In [4]:
blosc_settings = defaultdict(lambda: {})
for ctype in ['lz4', 'lz4hc', 'zlib', 'zstd']:
    for shuffle,shufflep in [('shuffle', 1), ('bitshuffle', 2)]:
        for clevel in [3,5,8]: #[3,5,8]:
            key = f'BLOSC_{clevel}_{shuffle}_{ctype}'
            toadd = {**blosc_settings[key], **hdf5plugin.Blosc(cname=ctype, clevel=clevel, shuffle=shufflep)}
            blosc_settings[key] = toadd
            
gzip_settings = defaultdict( lambda: {'compression':'gzip'})
for shuffle,shufflep in [(None, False), ('shuffle', True)]:
    for clevel in [3,5,8]: #[3,5,8]:
        key = f'GZIP_{clevel}_{shuffle}'
        toadd = {**gzip_settings[key], 'shuffle':shufflep, 'compression_opts':clevel}
        gzip_settings[key] = toadd

settings = {
    'Uncompressed':{},
    'LZF': {'compression':'lzf'},
    'LZF Shuffle': {'compression':'lzf', 'shuffle':True},
    'LZ4': {**hdf5plugin.LZ4(nbytes=0)},
    **{f'ZSTD_{cl}':{**hdf5plugin.Zstd(clevel=cl)} for cl in [3, 10, 15, 20, 22]},
    **gzip_settings,
    **blosc_settings
}
compression_methods = settings

In [5]:
def test_compression(img, params={}, cs=(32,32,32)):
    out = []
    with io.BytesIO() as tmpfile:
        with h5py.File(tmpfile, 'w') as h5file:
            start = time.time()
            h5file.create_dataset("data", data=img, **params, chunks=cs)
            out.append(tmpfile.getbuffer().nbytes)
        out.append( time.time() - start )
        start = time.time()
        with h5py.File(tmpfile, 'r') as h5file:
            tmp = np.array( h5file['data'] )
        out.append( time.time() - start )
    return out

In [6]:
N = 10
chunk_size = 256
subsampled_images = defaultdict(lambda: [])
for dname,dset in tqdm.tqdm(datasets.items()):
    for i in range(N):
        selector = None
        repeat = True
        while repeat:
            if dname == 'visium':
                x = np.random.randint(0,dset.shape[0]-chunk_size)
                y = np.random.randint(0,dset.shape[1]-chunk_size)
                selector = [
                    slice(x,x+chunk_size),
                    slice(y,y+chunk_size),
                    slice(None)
                ]
            elif len(dset.shape) == 4: #3d image multichannel
                z = np.random.randint(0,max(1,dset.shape[0]-chunk_size))
                x = np.random.randint(0,dset.shape[2]-chunk_size)
                y = np.random.randint(0,dset.shape[3]-chunk_size)
                selector = [
                    slice(z,z+chunk_size),
                    slice(None),
                    slice(x,x+chunk_size),
                    slice(z,z+chunk_size),
                ]
                if dset.shape[0] < chunk_size:
                    selector[0] = slice(None)
            else: #3d image single channel
                z = np.random.randint(0,max(1,dset.shape[0]-chunk_size))
                x = np.random.randint(0,dset.shape[1]-chunk_size)
                y = np.random.randint(0,dset.shape[2]-chunk_size)
                selector = [
                    slice(z,z+chunk_size),
                    slice(x,x+chunk_size),
                    slice(y,y+chunk_size)
                ]
                if dset.shape[0] < chunk_size:
                    selector[0] = slice(None)
                
            tmp = np.array(dset[tuple(selector)])
            
            if dname == 'fMOST' and tmp.min() == 0:
                repeat = True
            else:
                repeat = False
                subsampled_images[dname].append(tmp)

100%|██████████| 9/9 [01:23<00:00,  9.27s/it]


In [7]:
data = pd.DataFrame(
    {
        'Dataset':[],
        'Compression Method':[],
        'Compressed Size':[],
        'Read Time':[],
        'Write Time':[]
    }
)

benchmark_start = time.time()
last_start = benchmark_start

pbar = tqdm.tqdm(total=N*len(subsampled_images)*len(compression_methods), ncols=80, ascii=True)
for mname,mparam in compression_methods.items():
    for dname,dset in subsampled_images.items():
        for img in dset:
            cs = [min(32,x) for x in img.shape]
            try:
                s,wt,rt = test_compression(img, params=mparam, cs=tuple(cs))
            except:
                print(mname, dname, img.shape)
            
            data = data.append({
                'Dataset':dname,
                'Compression Method':mname,
                'Compressed Size':s,
                'Read Time':rt,
                'Write Time':wt
            }, ignore_index=True)
            
            pbar.update(1)
            
print("Benchmark ended at", time.time() - benchmark_start)

100%|#####################################| 3510/3510 [2:01:08<00:00,  2.98it/s]

Benchmark ended at 7268.172533750534


In [8]:
means = data.groupby(['Dataset', 'Compression Method']).mean()
data['Compression Ratio'] = data.apply(
    lambda x: means.loc[x['Dataset']].loc['Uncompressed']['Compressed Size'] / x['Compressed Size'],
    axis=1
)

data['Effective Compression Rate (MB/s)'] = data.apply(
    lambda x: means.loc[x['Dataset']].loc['Uncompressed']['Compressed Size'] / (10**6) / x['Write Time'],
    axis=1
)

data['Effective Decompression Rate (MB/s)'] = data.apply(
    lambda x: means.loc[x['Dataset']].loc['Uncompressed']['Compressed Size'] / (10**6) / x['Read Time'],
    axis=1
)

data.sort_values(by=['Compression Ratio'], ascending=False).to_csv('benchmark.csv')

In [9]:
data.sort_values(by=['Effective Compression Rate (MB/s)'], ascending=False).head(n=50)

,Dataset,Compression Method,Compressed Size,Read Time,Write Time,Compression Ratio,Effective Compression Rate (MB/s),Effective Decompression Rate (MB/s)
40,noise16,Uncompressed,16797432.0,0.011856,0.009047,1.000000,1856.678865,1416.806488
26,fly,Uncompressed,83909416.0,0.049139,0.052590,1.000000,1595.535383,1707.608850
20,fly,Uncompressed,83909416.0,0.049354,0.052766,1.000000,1590.222122,1700.168109
29,fly,Uncompressed,83909416.0,0.049311,0.052780,1.000000,1589.805484,1701.639546
22,fly,Uncompressed,83909416.0,0.049257,0.052831,1.000000,1588.270119,1703.492736
27,fly,Uncompressed,83909416.0,0.049931,0.052832,1.000000,1588.241449,1680.497735
28,fly,Uncompressed,83909416.0,0.049244,0.052840,1.000000,1587.997794,1703.938102
21,fly,Uncompressed,83909416.0,0.049378,0.052848,1.000000,1587.747051,1699.338979
23,fly,Uncompressed,83909416.0,0.049456,0.053004,1.000000,1583.083383,1696.635569
25,fly,Uncompressed,83909416.0,0.049463,0.053012,1.000000,1582.827071,1696.422938
